In [1]:
#Importing Libraries

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.image as mpimg
import time

**Here we use two Models for prediction.
Model1 predicts the damage detection and Model2 predicts the extent_of_damage**

In [2]:
#Importing all the training images to the input X from Train folder

train_image = []
for i in tqdm(range(801)):
    img = image.load_img('Train/Train/' + str(i) + '.jpeg',target_size=(224,224,3))  ## link to download Train Dataset Folder
    #https://drive.google.com/drive/folders/18_3G_c49punpzjPvnhNB0DN-RAwkZfLE?usp=sharing
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
print("The shape of input X is",X.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 801/801 [00:37<00:00, 21.08it/s]


The shape of input X is (801, 224, 224, 3)


In [3]:
#Importing the excel file and creating dummies for class column and saving it to target Y1 for damage detection and Y2 for extent_of_damatge

df = pd.read_excel("Train/training_label.xlsx")
df = df.drop(["image","subset"],axis=1)
df["class"]=df["class"].str.replace(" ","")
y1=df['class'].str.get_dummies(sep=',')
y2=pd.get_dummies(df["extent_of_damage"])
class_names=y1.columns
print("The shape of target output Y1 is",y1.shape)
print("\nThe first five rows of Y1 is as follows:\n",y1.head())
print("\nThe shape of target output Y2 is",y2.shape)
print("\nThe first five rows of Y2 is as follows:\n",y2.head())

The shape of target output Y1 is (801, 6)

The first five rows of Y1 is as follows:
    dent  glass_shatter  head_lamp  scratch  tail_lamp  unknown
0     1              0          1        0          0        0
1     0              0          1        0          0        0
2     0              0          0        1          0        0
3     0              0          1        0          0        0
4     0              0          0        0          0        1

The shape of target output Y2 is (801, 4)

The first five rows of Y2 is as follows:
    0  1  2  3
0  0  0  0  1
1  0  1  0  0
2  0  1  0  0
3  0  1  0  0
4  1  0  0  0


In [4]:
#Importing the test images to test_x from Test folder

test_image = []
for i in tqdm(range(801,1101)):
    img = image.load_img('Test/Test/' + str(i) + '.jpeg',target_size=(224,224,3))# link to download Test Dataset
    #https://drive.google.com/drive/folders/1JdLukt1x4LgIZHxZ3n0LDUSs95oTe2tO?usp=sharing
    img = image.img_to_array(img)
    img = img/255
    
    test_image.append(img)
test_x = np.array(test_image)
print("The shape of test_x is",test_x.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:17<00:00, 17.06it/s]

The shape of test_x is (300, 224, 224, 3)


In [5]:
#Spliting the data into 90% train data and 10% valid data for model1

X_train1, X_valid1, y_train1, y_valid1 = train_test_split(X, y1, random_state=42, test_size=0.1)

In [6]:
#Loading the pre-trained model1 for damage detection

from keras.models import load_model
model1 = load_model('model1.h5')#link to download pretrained model
#https://drive.google.com/file/d/1R-JCX4s9qQulMTVnxR7aDxCeX0RVEaw5/view?usp=sharing

In [7]:
#Evaluating the loss and accuracy on train data for model1

train_loss1, train_accuracy1 = model1.evaluate(X_train1,y_train1)
print("\nFor Model1, the training loss is ",train_loss1,"and the training accuracy is ",train_accuracy1)

23/23 [==============================] - 123s 4s/step - loss: 0.0017 - accuracy: 0.9403

For Model1, the training loss is  0.0017395718023180962 and the training accuracy is  0.9402777552604675


In [8]:
#Evaluating the loss and accuracy on validation data for model1

validation_loss1, validation_accuracy1 = model1.evaluate(X_valid1,y_valid1)
print("\nFor Model1, the training loss is ",validation_loss1,"and the training accuracy is ",validation_accuracy1)

3/3 [==============================] - 18s 3s/step - loss: 0.4245 - accuracy: 0.7407

For Model1, the training loss is  0.4245094060897827 and the training accuracy is  0.7407407164573669


In [9]:
#Using the pre-trained model1 to carry out prediction on unseen test data

start = time.time()
pred1=model1.predict(test_x)
end = time.time()
time1=end-start
print("Time taken to predict:",time1,"seconds")

10/10 [==============================] - 60s 4s/step
Time taken to predict: 85.3092942237854 seconds


In [10]:
#Setting the threshold and creating the output dataframe

pred1[pred1>=0.5]=1
pred1[pred1<0.5]=0
pred1 = pred1.astype(int)
pred1_df = pd.DataFrame(pred1,columns=class_names)
pred1_df.index = np.arange(1, len(pred1)+1)
first_column="image/" + (pred1_df.index + 800).astype(str)+".jpeg"
pred1_df.insert(0, 'image', first_column)
print("The prediction for damage detection is as follows:\n")
print(pred1_df)

The prediction for damage detection is as follows:

               image  dent  glass_shatter  head_lamp  scratch  tail_lamp  \
1     image/801.jpeg     0              0          0        0          1   
2     image/802.jpeg     0              0          0        0          0   
3     image/803.jpeg     0              0          0        0          0   
4     image/804.jpeg     1              0          0        1          0   
5     image/805.jpeg     1              0          1        0          0   
..               ...   ...            ...        ...      ...        ...   
296  image/1096.jpeg     0              0          0        0          0   
297  image/1097.jpeg     0              1          0        0          0   
298  image/1098.jpeg     0              0          1        0          0   
299  image/1099.jpeg     1              0          0        0          0   
300  image/1100.jpeg     0              0          0        0          1   

     unknown  
1          0  
2    

In [11]:
#Now we use Model2 to predict extent of damage

from keras.models import load_model
model2 = load_model('model2.h5') # link to download pretrained model
#https://drive.google.com/file/d/12pnHm9MEuw3prpRZRur6ddN9iV3vc8Vy/view?usp=sharing

In [12]:
#Spliting the data into 90% train data and 10% valid data for model2

X_train2, X_valid2, y_train2, y_valid2 = train_test_split(X, y2, random_state=42, test_size=0.1)

In [13]:
#Evaluating the loss and accuracy on train data for model2

train_loss2, train_accuracy2 = model2.evaluate(X_train2,y_train2)
print("\nFor Model2, the training loss is ",train_loss2,"and the training accuracy is ",train_accuracy2)

23/23 [==============================] - 144s 4s/step - loss: 0.0495 - accuracy: 0.9819

For Model2, the training loss is  0.04953427240252495 and the training accuracy is  0.9819444417953491


In [14]:
#Evaluating the loss and accuracy on validation data for model2

validation_loss2, validation_accuracy2 = model2.evaluate(X_valid2,y_valid2)
print("\nFor Model1, the training loss is ",validation_loss2,"and the training accuracy is ",validation_accuracy2)

3/3 [==============================] - 22s 4s/step - loss: 0.6059 - accuracy: 0.8395

For Model1, the training loss is  0.6058608293533325 and the training accuracy is  0.8395061492919922


In [15]:
#Using the pre-trained model2 to carry out prediction on unseen test data

start = time.time()
pred2=model2.predict(test_x)
end = time.time()
time2=end-start
print("Time taken to predict:",time2,"seconds\n")
pred2 = np.argmax(pred2, axis=1)
print("\nThe prediction of extent_of_damage is as follows:\n",pred2)

10/10 [==============================] - 114s 4s/step
Time taken to predict: 118.24885129928589 seconds


The prediction of extent_of_damage is as follows:
 [1 1 0 1 3 1 1 0 1 1 1 1 1 1 1 1 1 1 0 3 2 0 1 0 1 0 0 1 1 2 1 1 1 1 2 1 1
 2 0 1 0 2 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 2 1 1 2 1 3 1 1 2 1 1 1 0 0 1 1
 1 1 0 3 1 0 1 1 1 1 1 0 2 0 1 1 1 2 1 1 2 1 1 2 1 1 1 2 1 0 1 1 1 0 1 0 1
 0 0 1 1 1 1 0 3 1 1 1 1 0 1 1 0 0 0 1 0 1 1 1 3 0 0 3 0 2 1 0 1 0 1 1 1 1
 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 2 3 1 2 1 1 0 1 0 3 1 1 1 3
 1 1 1 2 0 1 0 2 2 1 1 1 1 0 1 1 2 0 1 1 1 2 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 1 2 0 1 1 1 3 2 1 1 2 1 0 1 1 1 3 1 1 1 1 0 1 1 1 0 3 0 1 1 1 0 1 1 0
 1 3 0 3 0 1 2 0 1 1 2 3 1 1 1 1 0 3 1 1 0 2 1 0 1 1 1 1 1 1 1 2 1 1 0 1 0
 2 2 1 1]


In [16]:
#Combining both predictions and  creating final prediction

pred1_df["extent_of_damage"]=pred2
final_pred_df = pred1_df
print("The final prediction is as follows:\n")
print(final_pred_df)

The final prediction is as follows:

               image  dent  glass_shatter  head_lamp  scratch  tail_lamp  \
1     image/801.jpeg     0              0          0        0          1   
2     image/802.jpeg     0              0          0        0          0   
3     image/803.jpeg     0              0          0        0          0   
4     image/804.jpeg     1              0          0        1          0   
5     image/805.jpeg     1              0          1        0          0   
..               ...   ...            ...        ...      ...        ...   
296  image/1096.jpeg     0              0          0        0          0   
297  image/1097.jpeg     0              1          0        0          0   
298  image/1098.jpeg     0              0          1        0          0   
299  image/1099.jpeg     1              0          0        0          0   
300  image/1100.jpeg     0              0          0        0          1   

     unknown  extent_of_damage  
1          0     

In [19]:
#Exporting the prediction to excel file

final_pred_df.to_excel('prediction.xlsx',index=False)

In [20]:
#Summary of both Models

print("For Model1 i.e damage_detection:\nTrain accuracy =",train_accuracy1,"\nValidation accuracy =",validation_accuracy1,"\nTime taken =",time1,"seconds")
print("\n")
print("For Model2 i.e extent_of_damage prediction:\nTrain accuracy =",train_accuracy2,"\nValidation accuracy =",validation_accuracy2,"\nTime taken =",time2,"seconds")

For Model1 i.e damage_detection:
Train accuracy = 0.9402777552604675 
Validation accuracy = 0.7407407164573669 
Time taken = 85.3092942237854 seconds


For Model2 i.e extent_of_damage prediction:
Train accuracy = 0.9819444417953491 
Validation accuracy = 0.8395061492919922 
Time taken = 118.24885129928589 seconds
